# Import packages 


In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import DataFrame, concat
import pandas as pd
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()



# Load the data

In [443]:
file_path = r'D:\project-rice-yield-prediction\datasets and notes\BD-rice(ok).xlsx'
df= pd.read_excel(file_path,sheet_name='Aman',skiprows=1)

In [444]:
# Display the first few rows of the dataframe
print(df.head())

   rice  year  district weather district division     yield   precip_1  \
0  Aman  2006  Bagerhat           Khulna   Khulna  0.728570  16.838710   
1  Aman  2007  Bagerhat           Khulna   Khulna  0.522331  19.064516   
2  Aman  2008  Bagerhat           Khulna   Khulna  0.663455   9.709677   
3  Aman  2009  Bagerhat           Khulna   Khulna  0.769729  11.193548   
4  Aman  2010  Bagerhat           Khulna   Khulna  0.710355   5.806452   

    precip_2  precip_5  precip_6  temp_1  temp_2  temp_5  temp_6  
0  11.741935  0.033333  0.000000    28.9    28.7    24.1    20.2  
1   5.161290  3.766667  0.000000    28.6    29.3    24.2    19.1  
2   6.516129  0.000000  0.000000    28.5    29.0    24.1    20.6  
3  18.322581  0.666667  0.000000    29.0    29.0    24.5    19.3  
4   6.612903  0.000000  0.419355    29.6    29.7    25.1    19.3  


In [445]:
# Filter out the yield data
yield_data = df[df['yield'].notnull()]

In [446]:
# Show filtered data
yield_data.head()

,rice,year,district,weather district,division,yield,precip_1,precip_2,precip_5,precip_6,temp_1,temp_2,temp_5,temp_6
0,Aman,2006,Bagerhat,Khulna,Khulna,0.728570,16.838710,11.741935,0.033333,0.000000,28.9,28.7,24.1,20.2
1,Aman,2007,Bagerhat,Khulna,Khulna,0.522331,19.064516,5.161290,3.766667,0.000000,28.6,29.3,24.2,19.1
2,Aman,2008,Bagerhat,Khulna,Khulna,0.663455,9.709677,6.516129,0.000000,0.000000,28.5,29.0,24.1,20.6
3,Aman,2009,Bagerhat,Khulna,Khulna,0.769729,11.193548,18.322581,0.666667,0.000000,29.0,29.0,24.5,19.3
4,Aman,2010,Bagerhat,Khulna,Khulna,0.710355,5.806452,6.612903,0.000000,0.419355,29.6,29.7,25.1,19.3


In [447]:
# Group the data by 'district' and sum the 'yield' for each district
top_10_districts = pd.DataFrame(yield_data.groupby('district')['yield'].sum()).sort_values(by='yield', ascending=False).reset_index()

# Get the names of the top 10 districts
top_10_district_names = top_10_districts['district'][:10]
top_10_district_names


0    Khagrachari
1     Chittagong
2      Rangamati
3      Bandarban
4        Kushtia
5       Rajshahi
6        Jessore
7        Rangpur
8    Kishoregonj
9          Bogra
Name: district, dtype: object

In [448]:
top_10_districts.head(10)

,district,yield
0,Khagrachari,30.810699
1,Chittagong,30.745561
2,Rangamati,30.421433
3,Bandarban,30.279954
4,Kushtia,28.174365
5,Rajshahi,28.035529
6,Jessore,27.570542
7,Rangpur,27.284379
8,Kishoregonj,26.891896
9,Bogra,26.483879


# Data Standardization

In [449]:
# Normalize the data: Drop the target variable 'yield' and categorical columns
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(yield_data.drop(columns=['rice', 'year', 'district', 'weather district', 'division', 'yield']))

# Create a DataFrame for the scaled data
scaled_df = pd.DataFrame(data=scaled, columns=yield_data.drop(columns=['rice', 'year', 'district', 'weather district', 'division', 'yield']).columns)

In [450]:
# Check the normalized data
scaled_df.head()

,precip_1,precip_2,precip_5,precip_6,temp_1,temp_2,temp_5,temp_6
0,0.293917,0.245844,0.002740,0.000000,0.617647,0.484848,0.681818,0.571429
1,0.341080,0.067367,0.309589,0.000000,0.529412,0.666667,0.693182,0.428571
2,0.142857,0.104112,0.000000,0.000000,0.500000,0.575758,0.681818,0.623377
3,0.174299,0.424322,0.054795,0.000000,0.647059,0.575758,0.727273,0.454545
4,0.060150,0.106737,0.000000,0.067358,0.823529,0.787879,0.795455,0.454545


In [451]:
# Create a copy of the original DataFrame
yield_data_scaled = yield_data.copy()

# Replace the relevant columns with the scaled values
yield_data_scaled[scaled_df.columns] = np.array(scaled_df)

# If you want to create a new column 'district_copy' based on the 'district' column
yield_data_scaled['districts'] = list(yield_data_scaled['district'])


In [452]:
# Set the index of the DataFrame to 'district' and 'year'
yield_data_scaled = yield_data_scaled.set_index(['district', 'year'])

# Display the first few rows of the updated DataFrame
yield_data_scaled.head()


rice weather district division     yield  precip_1  precip_2  \
district year                                                                 
Bagerhat 2006  Aman           Khulna   Khulna  0.728570  0.293917  0.245844   
         2007  Aman           Khulna   Khulna  0.522331  0.341080  0.067367   
         2008  Aman           Khulna   Khulna  0.663455  0.142857  0.104112   
         2009  Aman           Khulna   Khulna  0.769729  0.174299  0.424322   
         2010  Aman           Khulna   Khulna  0.710355  0.060150  0.106737   

               precip_5  precip_6    temp_1    temp_2    temp_5    temp_6  \
district year                                                               
Bagerhat 2006  0.002740  0.000000  0.617647  0.484848  0.681818  0.571429   
         2007  0.309589  0.000000  0.529412  0.666667  0.693182  0.428571   
         2008  0.000000  0.000000  0.500000  0.575758  0.681818  0.623377   
         2009  0.054795  0.000000  0.647059  0.575758  0.727273  0.454545   
         2010  0.000000  0.067358  0.823529  0.787879  0.795455  0.454545   

              districts  
district year            
Bagerhat 2006  Bagerhat  
         2007  Bagerhat  
         2008  Bagerhat  
         2009  Bagerhat  
         2010  Bagerhat

#  Train Test Split

In [477]:
# Function to split the data into train and test sets
def train_test_split(data):
    # Calculate the size of the training set (80% of the data)
    size = int(len(data) * 0.8)

    # For train data, collect 80% of the data, dropping the target variable 'yield'
    x_train = data.drop(columns=['yield']).iloc[0:size]
    
    # For test data, collect the remaining 20% of the data
    x_test = data.drop(columns=['yield']).iloc[size:]

    # Define the target variable for training and testing
    y_train = data['yield'].iloc[0:size]
    y_test = data['yield'].iloc[size:]

    return x_train, x_test, y_train, y_test


In [478]:

districts=list(set(yield_data_scaled.districts))
# Initialize empty lists for training and testing data
X_train = []
X_test = []
Y_train = []
Y_test = []
for i in range(0,len(districts)):
    data=yield_data_scaled[yield_data_scaled['districts']==districts[i]]
    # applied the function I created above
    x_train, x_test,y_train,y_test=train_test_split(data) 
    X_train.append(x_train)
    X_test.append(x_test)
    Y_train.append(y_train)
    Y_test.append(y_test)

In [479]:

X_train[0].head()


rice weather district division  precip_1  precip_2  precip_5  \
district year                                                                 
Dhaka    1993  Aman            Dhaka    Dhaka  0.224880  0.305337  0.052055   
         1994  Aman            Dhaka    Dhaka  0.041695  0.142607  0.038356   
         1995  Aman            Dhaka    Dhaka  0.179084  0.242345  0.306849   
         1996  Aman            Dhaka    Dhaka  0.112782  0.243220  0.000000   
         1997  Aman            Dhaka    Dhaka  0.312372  0.128609  0.002740   

               precip_6    temp_1    temp_2    temp_5    temp_6 districts  
district year                                                              
Dhaka    1993  0.000000  0.529412  0.393939  0.636364  0.519481     Dhaka  
         1994  0.000000  0.705882  0.575758  0.613636  0.415584     Dhaka  
         1995  0.005181  0.529412  0.606061  0.659091  0.415584     Dhaka  
         1996  0.000000  0.617647  0.363636  0.602273  0.506494     Dhaka  
         1997  0.113990  0.558824  0.636364  0.670455  0.415584     Dhaka

In [456]:

Y_train[0].head()

district  year
Dhaka     1993    0.529201
          1994    0.513570
          1995    0.463499
          1996    0.569768
          1997    0.529152
Name: yield, dtype: float64

In [480]:
# Concatenate each train dataset in X_train and Y_train lists
X_train = pd.concat(X_train) if isinstance(X_train, list) else X_train
Y_train = pd.concat(Y_train) if isinstance(Y_train, list) else Y_train

# Concatenate each test dataset in X_test and Y_test lists
X_test = pd.concat(X_test) if isinstance(X_test, list) else X_test
Y_test = pd.concat(Y_test) if isinstance(Y_test, list) else Y_test

In [481]:
# double check the train data and test data and our target 
X_train.head()

rice weather district division  precip_1  precip_2  precip_5  \
district year                                                                 
Dhaka    1993  Aman            Dhaka    Dhaka  0.224880  0.305337  0.052055   
         1994  Aman            Dhaka    Dhaka  0.041695  0.142607  0.038356   
         1995  Aman            Dhaka    Dhaka  0.179084  0.242345  0.306849   
         1996  Aman            Dhaka    Dhaka  0.112782  0.243220  0.000000   
         1997  Aman            Dhaka    Dhaka  0.312372  0.128609  0.002740   

               precip_6    temp_1    temp_2    temp_5    temp_6 districts  
district year                                                              
Dhaka    1993  0.000000  0.529412  0.393939  0.636364  0.519481     Dhaka  
         1994  0.000000  0.705882  0.575758  0.613636  0.415584     Dhaka  
         1995  0.005181  0.529412  0.606061  0.659091  0.415584     Dhaka  
         1996  0.000000  0.617647  0.363636  0.602273  0.506494     Dhaka  
         1997  0.113990  0.558824  0.636364  0.670455  0.415584     Dhaka

In [460]:
X_train.shape

(1021, 12)

# Data Preprocessing

# convert series to supervised learning

In [504]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

# create the reshape function
def reshape_data(train,test):
    #Frame as supervised learning and drop all time t columns except
    reframed_train = series_to_supervised(train, 1, 1)
    reframed_test = series_to_supervised(test, 1, 1)
    # split into train and test sets
    train= reframed_train.values
    test=reframed_test.values
    # split into input and outputs
    train_X, y_train = train[:, :-1], train[:, -1]
    test_X, y_test = test[:, :-1], test[:, -1]
    # reshape input to be 3D [samples, timesteps, features]
    x_train = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    x_test = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    return x_train,x_test,y_train,y_test

In [506]:
encoder = LabelEncoder()
#combine X train and Y train as train data 
train_data=pd.DataFrame()
train_data[X_train.columns]=X_train
train_data[Y_train.columns]=Y_train
train_data['districts']= encoder.fit_transform(train_data['districts'])
#combine X test and Y test as test data 
test_data=pd.DataFrame()
test_data[X_test.columns]=X_test
test_data[Y_test.columns]=Y_test
test_data['districts']= encoder.fit_transform(test_data['districts'])

In [507]:
train_data

rice weather district  division  precip_1  precip_2  precip_5  \
district year                                                                  
Dhaka    1993  Aman            Dhaka     Dhaka  0.224880  0.305337  0.052055   
         1994  Aman            Dhaka     Dhaka  0.041695  0.142607  0.038356   
         1995  Aman            Dhaka     Dhaka  0.179084  0.242345  0.306849   
         1996  Aman            Dhaka     Dhaka  0.112782  0.243220  0.000000   
         1997  Aman            Dhaka     Dhaka  0.312372  0.128609  0.002740   
...             ...              ...       ...       ...       ...       ...   
Natore   2013  Aman         Rajshahi  Rajshahi  0.006152  0.149606  0.000000   
         2014  Aman         Rajshahi  Rajshahi  0.102529  0.241470  0.000000   
         2015  Aman         Rajshahi  Rajshahi  0.178401  0.038495  0.016438   
         2016  Aman         Rajshahi  Rajshahi  0.194122  0.074366  0.000000   
         2017  Aman         Rajshahi  Rajshahi  0.205742  0.118985  0.013699   

               precip_6    temp_1    temp_2    temp_5    temp_6  districts  \
district year                                                                
Dhaka    1993  0.000000  0.529412  0.393939  0.636364  0.519481         13   
         1994  0.000000  0.705882  0.575758  0.613636  0.415584         13   
         1995  0.005181  0.529412  0.606061  0.659091  0.415584         13   
         1996  0.000000  0.617647  0.363636  0.602273  0.506494         13   
         1997  0.113990  0.558824  0.636364  0.670455  0.415584         13   
...                 ...       ...       ...       ...       ...        ...   
Natore   2013  0.000000  0.882353  0.575758  0.420455  0.298701         43   
         2014  0.000000  0.852941  0.636364  0.409091  0.181818         43   
         2015  0.005181  0.647059  0.727273  0.568182  0.311688         43   
         2016  0.000000  0.617647  0.787879  0.511364  0.376623         43   
         2017  0.103627  0.558824  0.757576  0.477273  0.454545         43   

                  yield  
district year            
Dhaka    1993  0.529201  
         1994  0.513570  
         1995  0.463499  
         1996  0.569768  
         1997  0.529152  
...                 ...  
Natore   2013  1.096241  
         2014  1.059510  
         2015  1.196343  
         2016  1.026950  
         2017  1.001938  

[1021 rows x 13 columns]

# If the above code is not run then

In [496]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Assuming X_train and Y_train are defined

# Ensure Y_train is a DataFrame
if isinstance(Y_train, pd.Series):
    Y_train = Y_train.to_frame()

# Combine X_train and Y_train into a single DataFrame for training data
train_data = pd.DataFrame()
train_data[X_train.columns] = X_train
train_data[Y_train.columns] = Y_train  # Now Y_train should be a DataFrame

# If 'districts' is a column in your DataFrame, encode it
if 'districts' in train_data.columns:
    encoder = LabelEncoder()
    train_data['districts'] = encoder.fit_transform(train_data['districts'])

# Similarly, for test data
# Ensure Y_test is a DataFrame
if isinstance(Y_test, pd.Series):
    Y_test = Y_test.to_frame()

# Combine X_test and Y_test into a single DataFrame for test data
test_data = pd.DataFrame()
test_data[X_test.columns] = X_test
test_data[Y_test.columns] = Y_test  # Now Y_test should be a DataFrame

# Encode 'districts' if it exists in test_data
if 'districts' in test_data.columns:
    test_data['districts'] = encoder.fit_transform(test_data['districts'])

# Now train_data and test_data should be ready to use


In [509]:

# using the function to obtian reshaped x_train,x_test,y_train,y_test
x_train,x_test,y_train,y_test=reshape_data(train_data,test_data)

# Model 
## LSTM for Rice-yeild  data

In [3]:
X_train.shape

NameError: name 'X_train' is not defined

In [522]:
# Check the data types of x_train
print(X_train.dtypes)


rice                  int32
weather district      int32
division              int32
precip_1            float64
precip_2            float64
precip_5            float64
precip_6            float64
temp_1              float64
temp_2              float64
temp_5              float64
temp_6              float64
districts             int32
dtype: object


In [523]:
# Assuming X_train is a DataFrame and contains categorical columns
encoder = LabelEncoder()

# Apply LabelEncoder to all categorical columns
for column in X_train.select_dtypes(include=['object']).columns:
    X_train[column] = encoder.fit_transform(X_train[column])

# Now, convert to numpy array and to float32
x_train = np.array(X_train, dtype=np.float32)


In [524]:
# Assuming y_train is a DataFrame/Series
y_train = encoder.fit_transform(y_train)  # Encode y_train if it contains categorical labels
y_train = np.array(y_train, dtype=np.float32)  # Convert to float32 if necessary


In [525]:
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Data types in x_train:", X_train.dtypes)


Shape of x_train: (1021, 12)
Shape of y_train: (1012,)
Data types in x_train: rice                  int32
weather district      int32
division              int32
precip_1            float64
precip_2            float64
precip_5            float64
precip_6            float64
temp_1              float64
temp_2              float64
temp_5              float64
temp_6              float64
districts             int32
dtype: object


In [529]:

# Check for missing values (NaNs) in x_train
print("Missing values in x_train:", np.isnan(x_train).sum())
# Check for missing values (NaNs) in y_train
print("Missing values in y_train:", np.isnan(y_train).sum())



Missing values in x_train: 24
Missing values in y_train: 0


In [519]:
time_steps = 1  # You can adjust this value if needed
num_samples = x_train.shape[0]
num_features = x_train.shape[1]

# Reshape to (num_samples, time_steps, num_features)
x_train = x_train.reshape((num_samples, time_steps, num_features))

print("New shape of x_train:", x_train.shape)  # It should now be 3D


New shape of x_train: (1021, 1, 12)


In [520]:

# Create and compile the model
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))  # Assuming x_train is 3D
model.add(LSTM(60, activation='relu'))
model.add(Dense(1))  # Output layer
model.compile(loss='mae', optimizer='adam')

# Fit the model to the training data
history = model.fit(x_train, y_train, epochs=30, batch_size=50, verbose=1, shuffle=False)


ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1021
'y' sizes: 1012


In [282]:
print(history.history['loss'])


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
